#### Using PySpark to Train a Recommendation System for Movies

By: Matt Purvis

##### Imports

In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
# Create Spark Session
spark = SparkSession.builder.appName('rec').getOrCreate()

##### Import data and preview

In [0]:
data = spark.sql('select * from movielens_ratings_csv')

In [0]:
# Preview columns and dtypes
data.printSchema()

root
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)
-- userId: integer (nullable = true)

In [0]:
# Show the data
data.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
 12| 2.0| 0|
 15| 1.0| 0|
 17| 1.0| 0|
 19| 1.0| 0|
 21| 1.0| 0|
 23| 1.0| 0|
 26| 3.0| 0|
 27| 1.0| 0|
 28| 1.0| 0|
 29| 1.0| 0|
 30| 1.0| 0|
 31| 1.0| 0|
 34| 1.0| 0|
 37| 1.0| 0|
 41| 2.0| 0|
+-------+------+------+
only showing top 20 rows

In [0]:
# Describe data
data.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

##### Train Test Split

In [0]:
training, test = data.randomSplit([.8,.2])

##### Creating and fitting model

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol = 'userId', itemCol = 'movieId', ratingCol = 'rating')

In [0]:
model = als.fit(training)

##### Making Predictions

In [0]:
predictions = model.transform(test)

In [0]:
predictions.show()

+-------+------+------+-----------+
movieId|rating|userId| prediction|
+-------+------+------+-----------+
 31| 4.0| 12| 1.0316855|
 31| 1.0| 13| 0.9619879|
 31| 1.0| 24| 3.0384388|
 31| 1.0| 29| 1.4071256|
 85| 1.0| 5| 2.4567633|
 85| 4.0| 7| 5.259293|
 85| 1.0| 13| 0.5967778|
 65| 1.0| 16| 0.8879659|
 53| 2.0| 19| 2.1684813|
 78| 1.0| 2| 1.0409636|
 78| 1.0| 27| 0.51478934|
 78| 1.0| 28| -0.7665875|
 34| 1.0| 0| 0.3008244|
 34| 1.0| 14|0.065377146|
 28| 1.0| 5| 1.3695135|
 28| 2.0| 15| -2.0196717|
 28| 1.0| 17| 1.4255174|
 76| 3.0| 7| 4.117205|
 76| 1.0| 20| 2.3147316|
 26| 3.0| 0| 0.46096644|
+-------+------+------+-----------+
only showing top 20 rows

##### Evaluating the model

In [0]:
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'rating', predictionCol = 'prediction' )

In [0]:
rmse = evaluator.evaluate(predictions)

In [0]:
print('RMSE')
print(rmse)

RMSE
1.7935857619090028

The rmse is pretty bad considering ratings are on a scale of 1 to 5. The dataset was small and having more data would probably improve the model. Also more tweaking of the ALS Model parameters could be done to improve the model.

##### How to use the model to make predictions on new data

In [0]:
# Get a sample user
single_user = test.filter(test['userid'] == 11).select(['movieId','userId'])

In [0]:
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 21| 11|
 25| 11|
 39| 11|
 47| 11|
 48| 11|
 64| 11|
 72| 11|
 80| 11|
 90| 11|
 94| 11|
 99| 11|
+-------+------+

In [0]:
# Make predictions
recommendations = model.transform(single_user)

In [0]:
# Display predictions, ordering from best to worse
recommendations.orderBy('prediction',ascending = False).show()

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 25| 11| 4.5677156|
 47| 11| 3.3633826|
 21| 11| 2.7896516|
 48| 11| 2.0768685|
 64| 11| 1.7605971|
 39| 11| 1.5061889|
 90| 11| 1.1557848|
 99| 11| 0.9474633|
 72| 11| -0.3930569|
 94| 11|-0.67363125|
 80| 11| -3.216835|
+-------+------+-----------+

For this user we would highly recommend movie 25. We would not recommend movie 80 at all. We think they may moderately enjoy movie 47.